<h1>Atom Evaluation and Execution</h1>

In OpenCog, there are evaluatable and executable Atoms.

Some of them are <b>black-box</b> links (i.e. it's not clear what exactly it will just by reading the names of the Atoms) and some are <b>clear-box</b> links, e.g. `GreaterThanLink`, `EqualLink`, `AbsentLink` and `DeleteLink` etc.

We will explore both of them in this notebook.

<b>Note:</b> It's recommended to have some basic knowledge of atomese operations and pattern matching before running this notebook. See <b>atomese.ipynb</b> and <b>pattern_matching.ipynb</b> for details.

In [ ]:
import common
from opencog.utilities import initialize_opencog
from opencog.atomspace import AtomSpace, types
from opencog.type_constructors import *
from opencog.bindlink import execute_atom, evaluate_atom

atomspace = AtomSpace()
initialize_opencog(atomspace)

An example of a <b>clear-box</b> link:

In [ ]:
plus_link = PlusLink(NumberNode("1"), NumberNode("2"))
print(plus_link)

Use `execute_atom` to get the result:

In [ ]:
result = execute_atom(atomspace, plus_link)
print(result)

They can also be nested together:

In [ ]:
times_link = TimesLink(NumberNode("3"), NumberNode("0.5"))
greater_than_link = GreaterThanLink(plus_link, times_link)
print(greater_than_link)

This time, in order to get the result, we will have to use `evaluate_atom` instead since we are checking if the first number is greater than the second one:

In [ ]:
result = evaluate_atom(atomspace, greater_than_link)
print(result)

A <b>black-box</b> link is typically the one that will execute custom codes written in either <b>C++</b>, <b>Scheme</b>, or <b>Python</b>.

Since it's hard to tell what exactly the code will do before actually running it, it's basically a black-box for that reason.

Let's try to create one by firstly defining our own function:

In [ ]:
def check_tv(atom):
  val = atom.tv.mean * atom.tv.confidence
  if val > 0.5:
    return TruthValue(1, 1)
  else:
    return TruthValue(0, 1)

<b>Note:</b> It's expected to return a Truth Value for a evaluatable clause.

Now populate the AtomSpace with some knowledge:

In [ ]:
inh_1 = InheritanceLink(ConceptNode("Bob"), ConceptNode("cat"))
inh_2 = InheritanceLink(ConceptNode("Alice"), ConceptNode("dog"))

inh_1.tv = TruthValue(0.8, 0.5)
inh_2.tv = TruthValue(0.9, 0.7)

print("{}{}".format(inh_1, inh_2))

To turn it into an evaluatable link, we need to use an `EvaluationLink` and a `GroundedPredicateNode`.

The name of the `GroundedPredicateNode` that calls a custom function has to follow the following syntax:

<b>[written-language]: [function-name]</b>

In [ ]:
gpn = GroundedPredicateNode("py: check_tv")
print(gpn)

Give it a quick try and check the result:

In [ ]:
result = evaluate_atom(atomspace, EvaluationLink(gpn, ListLink(inh_1)))
print(result)

Similarity, to create an executable link, we can start with the custom function:

In [ ]:
def change_tv(atom):
  atom.tv = TruthValue(0.99, 0.99)
  return atom

<b>Note:</b> It's expected to return an Atom for a executable clause.

This time, we need to use an `ExecutionOutputLink` and a `GroundedSchemaNode` for creating an executable link.

The name of the `GroundedSchemaNode` should follow the same <b>[written-language]: [function-name]</b> syntax.

In [ ]:
gsn = GroundedSchemaNode("py: change_tv")
print(gsn)

Execute to get the result:

In [ ]:
result = execute_atom(atomspace, ExecutionOutputLink(gsn, ListLink(inh_2)))
print(result)

<h3>Exercise:</h3>

Given the additional knowledge below, try to use a `BindLink` to find the kind of <b>animal</b> that <b>Roy</b> inherits from, with the <b>confidence</b> of the `ConceptNode` representing that animal higher than <b>0.5</b>, and create a new `InheritanceLink` making `ConceptNode("Anna")` inherits from it, with a Truth Value <b>(stv 0.88 0.88)</b>.

In [ ]:
ConceptNode("cow").tv = TruthValue(0.5, 0.5)
ConceptNode("bird").tv = TruthValue(0.7, 0.8)
InheritanceLink(ConceptNode("Roy"), ConceptNode("cow"))
InheritanceLink(ConceptNode("Roy"), ConceptNode("bird"))
InheritanceLink(ConceptNode("cow"), ConceptNode("animal"))
InheritanceLink(ConceptNode("bird"), ConceptNode("animal"))
InheritanceLink(ConceptNode("Roy"), ConceptNode("virtual-assistant"))
InheritanceLink(ConceptNode("virtual-assistant"), ConceptNode("software"))

Let's start with the functions first, we need to have a function that will check if the Truth Value confidence of an Atom is higher than 0.5:

In [ ]:
def exr_check_tv(atom):
  if atom.tv.confidence > 0.5:
    return TruthValue(1, 1)
  else:
    return TruthValue(0, 1)

We can also have a function that will create the required `InheritanceLink` change its Truth Value:

<b>NOTE:</b> Technically it is not really needed to have a custom function for that. This is just for demostrating how an `ExecutionOutputLink` can be used in a `BindLink`.

In [ ]:
def exr_create(atom):
  inh_link = InheritanceLink(ConceptNode("Anna"), atom)
  inh_link.tv = TruthValue(0.88, 0.88)
  return inh_link

Now let's work on the `BindLink`, recalling:

```
BindLink
  variable-declarations (optional)
  pattern-to-be-matched
  instantiated-pattern
```

We only need one variable -- the target animal we are looking for:

In [ ]:
var_x = VariableNode("$x")
var_decl = TypedVariableLink(var_x, TypeNode("ConceptNode"))

The pattern is a little more complicated, we have to make sure that there is a mutual connection between <b>Roy</b> and <b>animal</b>:

In [ ]:
clause_1 = InheritanceLink(ConceptNode("Roy"), var_x)
clause_2 = InheritanceLink(var_x, ConceptNode("animal"))

print("{}{}".format(clause_1, clause_2))

...and has a Truth Value confidence higher than 0.5:

In [ ]:
clause_3 = EvaluationLink(GroundedPredicateNode("py: exr_check_tv"), ListLink(var_x))
print(clause_3)

Putting the clauses all together:

In [ ]:
pattern = AndLink(clause_1, clause_2, clause_3)
print(pattern)

For the last part -- rewriting, we'll have to call `exr_create` for creating the new `InheritanceLink`:

In [ ]:
rewrite = ExecutionOutputLink(GroundedSchemaNode("py: exr_create"), ListLink(var_x))
print(rewrite)

Putting them all together, the `BindLink` should look like this:

In [ ]:
bindlink = BindLink(var_decl, pattern, rewrite)
print(bindlink)

Finally, execute and check the result:

In [ ]:
result = execute_atom(atomspace, bindlink)
print(result)

Now you should be able to see the newly created `InheritanceLink`.